In [ ]:
!pip install psycopg2-binary

     |████████████████████████████████| 3.0 MB 12.8 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import psycopg2

# settings
datapath = '/Users/mirkoknoche/MIMIC/'
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,}'.format
%matplotlib inline

# database
conn_string = open('./intern/DB_connection.txt', 'r', encoding='UTF-8').read()

FileNotFoundError: [Errno 2] No such file or directory: './intern/DB_connection.txt'

# Selecting the patient group

We filter the patients in three steps.   
+ First we select from all admissions those who are **diagnosed with ARDS**:    
We find the diagnoses in the table DIAGNOSES_ICD. All diagnoses for each HADM_ID are listed there.    
ARDS is the abbreviation for "Acute Respiratory Distress Syndrome" and means a syndrome with multiple risk 
factors that trigger the acute onset of respiratory insufficiency.   
The ICD9 Code for ARDS is **51881**. So we select all HADM_IDs for which the number 51881  appeared in 
table DDIAGNOSES_ICD in column ICD9_CODE. We find 7497 unique HADM_ID with diagnosis ARDS.   
Because newborns with respiratory failure are diagnosed with ICD9 Code 769 
("Respiratory distress syndrome in newborn"), there is no need to separate the adults from the babies.
+ Then we examine which of these ARDS patients were **ventilated**. To do this, we check the table CPT_EVENTS 
(billing for procedures performed on patients) to see whether ventilation has been invoiced. 
We take into account the CPT (current procedural terminology =CPT) Codes 94003 and 94002. 
These numbers mean ventilation management.   
There are two other codes that deal with ventilation (94004 and 94660). But neither appear in this data.   
To be sure we can find all ventilated ARDS patients, we continue to review evidence of invasive ventilation.  
We look in the table PROCEDURES_ICD for the numbers 9672, 9671, 9670 in the column ICD9_Code. 
These numbers mean "Continuous invasive mechanical ventilation for 96 consecutive hours or more",
 "Continuous invasive mechanical ventilation for less then 96 consecutive hours", "Continuous invasive mechanical 
 ventilation of unspecified duration".      
We also look in the table PROCEDUREEVENTS_MV for the number 225792 in the column ITEMID.
This number means "Invasive Ventilation".   
We also look in the table DRGCODES for the numbers 1303, 475, 566, 576, 208 in the column DRG_CODE.
These numbers mean "Respiratory System Diagnosis with Ventilator Support".  
+ In the next step we exclude those patients who have never been extubated. These patients
 died before extubation or we cannot see the extubation or they were moved to another ICU . This are 778.


We find a total of 5897 HADM_IDs who met these criteria. They are saved in the file selection_hadm.



# Selecting the features

|Feature|Table|Column|Parameter|Matched on|Comment
|:-----------|:-----------|:---------|:--------------|:----------------|:-------------|
|**Demographics**| | | | | |.     
|Subject_ID|ADMISSIONS|SUBJECT_ID| | HADM_ID|               |   
|Gender|PATIENTS|GENDER| |SUBJECT_ID |    
|Age|PATIENTS| DOB| |SUBJECT_ID| Shifted Dates:  Calculate DOB - ADMITTIME| |  
|**Clinical**|  
|Number of Diagnoses|DIAGNOSES_ICD|SEQ_NUM||HADM_ID| Take the maximum|  
|Length of stay|ICU_STAYS|LOS| |HADM_ID |  |  
|ICU|ICU_STAYS|LAST_CAREUNIT| | HADM_ID| |  
|Tracheostomy|DRGCODES|DRG_CODE|44, 54 | HADM_ID|  |   
|**Bedside Signs** | 
|Heart Rate|CHARTEVENTS|ITEM_ID|211, 220045|HADM_ID|   
|Blood Pressure|CHARTEVENTS|ITEM_ID|52, 456, 225312, 220181, 220052| HADM_ID|  
|Temperatur|CHARTEVENTS|ITEM_ID|676, 677,223762|HADM_ID||
|O<sub>2</sub>  Saturation|CHARTEVENTS|ITEM_ID|646, 834, 22077, 22027|HADM_ID|  
|**Ventilator Set**|  
|Breath Rate| CHARTEVENTS|ITEM_ID|618, 220210, 224688|HADM_ID| |  
|Breath-Minute-Volume|	CHARTEVENTS	|ITEM_ID|	445, 448, 449, 224687, 1340, 1486, 1600| HADM_ID|  |
|FiO<sub>2</sub>|CHARTEVENTS|ITEM_ID| 189, 190, 3420, 3422, 223835| HADM_ID||
|Tidal Volume| CHARTEVENTS|ITEM_ID|681, 682, 683, 684, 224685, 224686, 224684    |HADM_ID| |   
|Mean Insp. Pressure| CHARTEVENTS|ITEM_ID|444, 224697|HADM_ID| |   
|PEEP|CHARTEVENTS|ITEM_ID|506, 224695|HADM_ID| |   
|**Blood Signs**|  
|PaO2|LABEVENTS|ITEM_ID|50821|HADM_ID| |   
|Base Excess|LABEVENTS|ITEM_ID|50802|HADM_ID| |   
|pH|LABEVENTS|ITEM_ID|50820|HADM_ID| |   
|Hemoglobin|LABEVENTS|ITEM_ID|51222|HADM_ID| |   
|Hematocrit|LABEVENTS|ITEM_ID|51221|HADM_ID| |   
|Kreatinin|LABEVENTS|ITEM_ID|50912|HADM_ID| |   
|Leukocytes (WBC)|CHARTEVENTS|ITEM_ID|1127, 861, 1542|HADM_ID| |   
|Lactic Acid|CHARTEVENTS|ITEM_ID|225668, 1531, 818|HADM_ID| |    



We select the measurement from the vital and blood values ​​that is closest to the time of extubation.

# Define label

We investigate the success of an extubation.  
Failed (0) = Re-intubated within 48 hours extubation.  
Succeeded (1) =  Remains extubated 48 hours after extubation

Direct information of whether the patient is on or off the ventilator at any given time is not available
 in MIMIC. To infer ventilation status, we used a heuristic approach, empirically utilizing information 
 from multiple chart variables that were informative of ventilation status:   
 Ventilator Mode, Minute Volume, Respiratory Pressure, Inspir. Pressure, Respiratory Support, 
 Plateau Pressure, APRV pressure, PEEP, High Pressure Relief, PVC, TCPCV, PSVlevel and Oxygen Delivery Device.    
 Ventilation status was coded as "MechVent" or "OxygenTherapy". The initiation of oxygen therapy indicates the 
 ventilation has ended.  
  An "extubation" event was defined as the time point where "MechVent" status changed from 1 to 0 
  and "OxygenTherapy" status changed from 0 to 1.   
Some patients whose data was recorded with carvue have the entry "Extubated" in the "Significant Events" 
section. This gives us the opportunity to check our determination of the time of extubation. 

## Find extubations

In [ ]:
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))

NameError: name 'conn_string' is not defined

In [ ]:
sql = """select i2.hadm_id, v.* 
        from ventsettings v
        inner join icustays i2 on i2.icustay_id = v.icustay_id 
        where (v.mechvent + v.oxygentherapy + v.extubated + v.selfextubated) > 0
        order by icustay_id, charttime """
df = pd.read_sql(sql, conn)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df.info()

Checking the extubations

In [ ]:
df_count_ex = df.groupby('icustay_id').extubated.sum()

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_count_ex)
plt.ylabel('Amount of ICU-stays')
plt.xlabel('Extubations')
plt.title('Histrogram of the extubations')
plt.savefig('./images/hist_extubations_per_icustay.png')
plt.show();

Many ICU stays have no explicit documented extubation. We need to extract the id's and find another way.

In [ ]:
err_ext = list(df_count_ex[df_count_ex == 0].index)
len(err_ext)

In [ ]:
sql = """select * from transfers"""
df_icuunit = pd.read_sql(sql, conn)

# Filtering the icustay_id's with no extubations
df_icuunit = df_icuunit[df_icuunit.icustay_id.isin(err_ext)]

# use only the interesting columns
df_icuunit = df_icuunit[['icustay_id', 'curr_careunit']]

# drop duplicates
df_icuunit.drop_duplicates(inplace=True)

In [ ]:
df_plot = df_icuunit.groupby('curr_careunit').count()

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
ax.bar(list(df_plot.index), height=list(df_plot.values.flatten()))
plt.ylabel('Missing extubations')
plt.xlabel('ICU units')
plt.title('Not documented extubations')
plt.savefig('./images/missing_extubations_per_unit.png')
plt.show();

Have a look on one patient.

In [ ]:
df_qry = df[df.icustay_id == 200063]
df_qry.head()

In [ ]:
# getting the changes
df_qry['extube'] = df_qry['mechvent'].diff()
df_qry['oxy'] = df_qry['oxygentherapy'].diff()

In [ ]:
df_qry.extube.fillna(0, inplace=True)
df_qry.oxy.fillna(0, inplace=True)

In [ ]:
df_qry.loc[160:180]

In [ ]:
df_qry[(df_qry['extube'] != 0) | (df_qry['oxy'] != 0)]

In [ ]:
# War die erste Extubation erfolgreich? 1 = Ja, 0 = Nein
select vel.hadm_id, vel."label" 
from vw_extubation_label vel 


In [ ]:
# Timestamp of extubation
select hadm_id, charttime 
from vw_timestamp_extubation vte

In [ ]:
# Timestamp of first extubation
select hadm_id, min(charttime)
from vw_timestamp_extubation vte
group by hadm_id 
order by hadm_id 

We dec

In [ ]:
conn.close()